<a href="https://colab.research.google.com/github/UmeshikaThathsarani/FuelO/blob/feature%2FlambdaFunctions/AWS_Lambda_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the trained model
model = keras.models.load_model('fuel_consumption_model.h5')

# Load the one-hot encoding and scaling transformers
with open('one_hot_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

def lambda_handler(event, context):
    # Parse the input parameters from the API Gateway event
    year = event['year']
    make = event['make']
    transmission = event['transmission']
    engine_cylinders = event['engineCylinders']
    turbocharger = event['turbocharger']
    fuel_type = event['fuelType']
    mpg = event['mpg']
    
    # Convert the input parameters to a numpy array
    input_data = np.array([[year, make, transmission, engine_cylinders, turbocharger, fuel_type, mpg]])
    
    # Perform one-hot encoding and scaling on the input data
    input_data_encoded = encoder.transform(input_data).toarray()
    input_data_scaled = scaler.transform(input_data_encoded)
    
    # Predict the fuel consumption using the trained model
    predictions = model.predict(input_data_scaled)
    city_fuel_consumption, highway_fuel_consumption = predictions[0]
    
    # Create the response body
    response_body = {
        'cityFuelConsumption': city_fuel_consumption,
        'highwayFuelConsumption': highway_fuel_consumption
    }
    
    # Create the response object
    response = {
        'statusCode': 200,
        'body': json.dumps(response_body)
    }
    
    # Return the response object
    return response
